# Segmenting and Clustering Neighborhoods in Toronto

In [4]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [5]:
wiki_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [22]:
r = requests.get(wiki_link)

soup = BeautifulSoup(r.text, 'html.parser')

#print(soup.prettify())

In [23]:
data = []
na='Not assigned'
table = soup.table
table_body = table.find('tbody')

headers=[h.get_text() for h in table_body.find_all('th')]
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    if len(cols)>1 and cols[1]!=na:
        data.append([ele for ele in cols if ele]) #
    
headers[-1]='Neighbourhood'
#data[0]=headers
del data[0]
data [:10]

[['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M5A', 'Downtown Toronto', 'Regent Park'],
 ['M6A', 'North York', 'Lawrence Heights'],
 ['M6A', 'North York', 'Lawrence Manor'],
 ['M7A', "Queen's Park", 'Not assigned'],
 ['M9A', 'Etobicoke', 'Islington Avenue'],
 ['M1B', 'Scarborough', 'Rouge'],
 ['M1B', 'Scarborough', 'Malvern'],
 ['M3B', 'North York', 'Don Mills North']]

In [8]:
df = pd.DataFrame(data, columns=headers)
df.head()

,Postcode,Borough,Neighbourhood
0,M4A,North York,Victoria Village
1,M5A,Downtown Toronto,Harbourfront
2,M5A,Downtown Toronto,Regent Park
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [32]:
#remove 'Not assigned' Neighborhoods

df.Neighbourhood.replace('Not assigned',df.Borough,inplace=True)

'Not assigned' in df['Neighbourhood'].unique()

False

Clean and drop duplicates 

In [12]:
test_df=df.copy()

test_df['Neighbourhood'] = test_df[['Postcode','Borough','Neighbourhood']].groupby(['Postcode','Borough'])['Neighbourhood'].transform(lambda x: ','.join(x))
test_df[['Postcode','Borough','Neighbourhood']].drop_duplicates(inplace=True)

test_df.head()


,Postcode,Borough,Neighbourhood
0,M4A,North York,Victoria Village
1,M5A,Downtown Toronto,"Harbourfront,Regent Park"
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M6A,North York,"Lawrence Heights,Lawrence Manor"


In [24]:
clean_df=test_df.drop_duplicates()
clean_df.reset_index(inplace=True, drop=True)
clean_df.head(20)

,Postcode,Borough,Neighbourhood
0,M4A,North York,Victoria Village
1,M5A,Downtown Toronto,"Harbourfront,Regent Park"
2,M6A,North York,"Lawrence Heights,Lawrence Manor"
3,M7A,Queen's Park,Not assigned
4,M9A,Etobicoke,Islington Avenue
5,M1B,Scarborough,"Rouge,Malvern"
6,M3B,North York,Don Mills North
7,M4B,East York,"Woodbine Gardens,Parkview Hill"
8,M5B,Downtown Toronto,"Ryerson,Garden District"
9,M6B,North York,Glencairn


### Final Dataframe

In [21]:
clean_df.shape

(102, 3)